# Data Preparation for Multiple Linear Regression

**Purpose:** Merge all datasets to create analysis-ready data for bias detection

**Inputs:**
- population_metrics_demo.csv (suitability scores)
- demographics.csv (ethnicity, sentence length, county)
- current_commits_clean.csv (offense details)
- selection_criteria.xlsx (offense severity tables)

**Output:**
- regression_analysis_data.csv (merged dataset ready for MLR)

## Step 1: Import Libraries

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

## Step 2: Set File Paths

In [5]:
# Define paths
data_dir = Path("../data")
outputs_dir = Path("../outputs")

..\data


## Step 3: Load Population Metrics (Suitability Scores)

In [6]:
# Load population metrics output
pop_metrics = pd.read_csv("C:\\Users\\gandh\\PycharmProjects\\PythonProject\\external_repos\\outputs\\population_metrics_demo.csv")

In [7]:
# Check shape and columns
pop_metrics.shape

(5000, 6)

In [8]:
pop_metrics.head()

,cdcno,desc_nonvio_curr,score,score_out_of,desc_nonvio_past,severity_trend
0,705626ffad,0.0,0.000000,1.0,NaN,NaN
1,015936e791,0.0,0.000000,1.0,NaN,NaN
2,8e3009d580,NaN,0.000000,0.0,NaN,NaN
3,043e1ac51d,NaN,0.000000,0.0,NaN,NaN
4,ffb452d91d,0.0,0.166667,3.0,0.0,0.5


## Step 4: Load Demographics (Ethnicity, Sentence Length)

In [9]:
# Load demographics
demographics = pd.read_csv(data_dir / "demographics.csv")

In [10]:
# Select relevant columns
demo_cols = [
    'cdcno',
    'ethnicity',
    'aggregate sentence in months',
    'controlling case sentencing county',
    'offense category',
    'time served in years'
]

demographics_subset = demographics[demo_cols]

In [11]:
demographics_subset.head()

,cdcno,ethnicity,aggregate sentence in months,controlling case sentencing county,offense category,time served in years
0,2cf2a233c4,Black,32,San Bernardino,Property Crimes,2.6
1,5a72696541,White,360,Sacramento,Crimes Against Persons,12.8
2,7d608b6a4c,White,300,Butte,Crimes Against Persons,15.2
3,39c1bc8c2f,Other,348,Butte,Property Crimes,27.5
4,220f2cdfc5,Black,312,Los Angeles,Crimes Against Persons,8.3


## Step 5: Load Current Commits (Offense Details)

In [13]:
# Load current commits with cleaned offense codes
current_commits = pd.read_csv(data_dir / "current_commitments_clean.csv")

C:\Users\gandh\AppData\Local\Temp\ipykernel_12136\580973997.py:2: DtypeWarning: Columns (7,22,23,24,25,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  current_commits = pd.read_csv(data_dir / "current_commitments_clean.csv")


In [14]:
# Select relevant columns
current_cols = [
    'cdcno',
    'offense_clean',
    'offense description',
    'offense category'
]

current_subset = current_commits[current_cols]

In [15]:
# Keep only the primary offense (first row per person)
# Assumption: First offense listed is the controlling offense
current_primary = current_subset.groupby('cdcno').first().reset_index()

In [16]:
current_primary.head()

,cdcno,offense_clean,offense description,offense category
0,00009164d5,212.5,Attempted Robbery 2nd,Crimes Against Persons
1,0000a5860b,314,Indecent Exposure,Crimes Against Persons
2,00015ca40d,273A,Abuse/Endanger Health of Child,Crimes Against Persons
3,0002afe811,187,Attempted Murder 1st,Crimes Against Persons
4,0003140564,666.5,Vehicle Theft w/ Prior Vehicle Related Theft C...,Property Crimes


## Step 6: Load Selection Criteria (Offense Severity Tables)

In [17]:
# Load selection criteria
selection = pd.read_excel(data_dir / "selection_criteria.xlsx", sheet_name='Penal codes')

In [18]:
# Clean offense codes in selection criteria (ensure matching format)
selection['Offenses'] = selection['Offenses'].astype(str).str.strip()

In [19]:
# Create lookup dictionary: offense_code -> table
offense_to_table = dict(zip(selection['Offenses'], selection['Table']))

# Create lookup dictionary: offense_code -> type
offense_to_type = dict(zip(selection['Offenses'], selection['Type']))

In [20]:
# Preview mappings
list(offense_to_table.items())[:10]

[('136.1', 'Table A'),
 ('210.5', 'Table A'),
 ('244', 'Table A'),
 ('245.2', 'Table A'),
 ('245.3', 'Table A'),
 ('246', 'Table A'),
 ('246.3', 'Table A'),
 ('262', 'Table A'),
 ('422', 'Table A'),
 ('451', 'Table A')]

## Step 7: Map Offense Codes to Severity Tables

In [21]:
# Add offense table classification to current commits
current_primary['offense_table'] = current_primary['offense_clean'].map(offense_to_table)
current_primary['offense_type'] = current_primary['offense_clean'].map(offense_to_type)

In [22]:
# Fill missing with "Other" (offenses not in any table)
current_primary['offense_table'].fillna('Other', inplace=True)
current_primary['offense_type'].fillna('Other', inplace=True)

C:\Users\gandh\AppData\Local\Temp\ipykernel_12136\4261455190.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  current_primary['offense_table'].fillna('Other', inplace=True)
C:\Users\gandh\AppData\Local\Temp\ipykernel_12136\4261455190.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [23]:
# Check distribution of offense tables
current_primary['offense_table'].value_counts()

offense_table
Other      59085
Table D     9775
Table A     9127
Table F     8022
Table E     6963
Table B     2504
Name: count, dtype: int64

## Step 8: Merge All Datasets

In [24]:
# Merge population metrics with demographics
analysis_data = pop_metrics.merge(
    demographics_subset,
    on='cdcno',
    how='inner'
)

In [25]:
# Merge with current commits (offense details)
analysis_data = analysis_data.merge(
    current_primary,
    on='cdcno',
    how='left',
    suffixes=('_demo', '_offense')
)

In [26]:
# Check merge results
analysis_data.shape

(5000, 16)

In [27]:
analysis_data.head()

,cdcno,desc_nonvio_curr,score,score_out_of,desc_nonvio_past,severity_trend,ethnicity,aggregate sentence in months,controlling case sentencing county,offense category_demo,time served in years,offense_clean,offense description,offense category_offense,offense_table,offense_type
0,705626ffad,0.0,0.000000,1.0,NaN,NaN,White,92,Madera,Crimes Against Persons,3.2,12022.1,Offense Committed While Released On Bail,Case Enhancement,Other,Other
1,015936e791,0.0,0.000000,1.0,NaN,NaN,Other,600,San Diego,Crimes Against Persons,5.3,29800,Poss/Own F'Arm by Felon or Addict,Other Crimes,Other,Other
2,8e3009d580,NaN,0.000000,0.0,NaN,NaN,American Indian,312,Shasta,Crimes Against Persons,15.8,290.018,Failure to Register,Crimes Against Persons,Other,Other
3,043e1ac51d,NaN,0.000000,0.0,NaN,NaN,Hispanic,24,Los Angeles,Crimes Against Persons,2.5,69,Resisting/Deterring Officer w/ Threat/Violence,Crimes Against Persons,Other,Other
4,ffb452d91d,0.0,0.166667,3.0,0.0,0.5,Hispanic,960,Los Angeles,Crimes Against Persons,12.1,667,Prior Felony Conviction of Serious Offense,Case Enhancement,Other,Other


## Step 9: Data Cleaning and Filtering

In [28]:
# Convert sentence to numeric (handle any errors)
analysis_data['aggregate sentence in months'] = pd.to_numeric(
    analysis_data['aggregate sentence in months'],
    errors='coerce'
)

In [29]:
# Drop rows with missing critical variables
analysis_data_clean = analysis_data.dropna(subset=[
    'aggregate sentence in months',
    'ethnicity',
    'score'
])

In [30]:
# Remove outliers (sentences > 50 years = 600 months)
# These are likely life sentences or data errors
analysis_data_clean = analysis_data_clean[
    (analysis_data_clean['aggregate sentence in months'] > 0) &
    (analysis_data_clean['aggregate sentence in months'] <= 600)
]

In [31]:
# Check final sample size
print(f"Original sample: {len(analysis_data):,}")
print(f"After cleaning: {len(analysis_data_clean):,}")
print(f"Dropped: {len(analysis_data) - len(analysis_data_clean):,}")

Original sample: 5,000
After cleaning: 4,165
Dropped: 835


## Step 10: Create Additional Variables

In [32]:
# Create binary variable: High suitability (score above median)
median_score = analysis_data_clean['score'].median()
analysis_data_clean['high_suitability'] = (analysis_data_clean['score'] > median_score).astype(int)

In [43]:
# Create categorical variable: Suitability tertiles (low/medium/high)
analysis_data_clean['suitability_category'] = pd.cut(
    analysis_data_clean['score'],
    bins=[0, 1, 2, 3],
    labels=['Low', 'Medium', 'High'],
    include_lowest=True
)

In [44]:
# Create binary variable: Violent offense (Table B or C)
analysis_data_clean['violent_offense'] = analysis_data_clean['offense_table'].isin(['Table B', 'Table C']).astype(int)

## Step 11: Descriptive Statistics

In [45]:
# Summary statistics for continuous variables
analysis_data_clean[[
    'aggregate sentence in months',
    'score',
    'desc_nonvio_curr',
    'desc_nonvio_past',
    'severity_trend'
]].describe()

,aggregate sentence in months,score,desc_nonvio_curr,desc_nonvio_past,severity_trend
count,4165.000000,4165.000000,2546.000000,1198.000000,687.000000
mean,201.404562,0.271366,0.296218,0.609106,0.492725
std,152.093065,0.382237,0.420836,0.445359,0.026397
min,16.000000,0.000000,0.000000,0.000000,0.454545
25%,72.000000,0.000000,0.000000,0.000000,0.474432
50%,180.000000,0.000000,0.000000,1.000000,0.500000
75%,300.000000,0.500000,0.666667,1.000000,0.500000
max,600.000000,1.000000,1.000000,1.000000,0.545455


In [46]:
# Frequency tables for categorical variables
print("Ethnicity Distribution:")
print(analysis_data_clean['ethnicity'].value_counts())
print()

print("Offense Table Distribution:")
print(analysis_data_clean['offense_table'].value_counts())
print()

print("Top 10 Counties:")
print(analysis_data_clean['controlling case sentencing county'].value_counts().head(10))

Ethnicity Distribution:
ethnicity
Hispanic            1621
Black               1055
White                759
Mexican              445
Other                141
American Indian       42
Filipino              28
Other Asian           25
Pacific Islander      12
Samoan                10
Puerto Rican           6
Vietnamese             5
Chinese                4
Laotian                3
Cuban                  2
Korean                 2
Hawaiian               1
Salvadorian            1
Indian                 1
Guatemalan             1
Cambodian              1
Name: count, dtype: int64

Offense Table Distribution:
offense_table
Other      2632
Table A     426
Table D     408
Table F     365
Table E     207
Table B     127
Name: count, dtype: int64

Top 10 Counties:
controlling case sentencing county
Los Angeles       1232
Riverside          328
San Diego          314
San Bernardino     283
Sacramento         192
Orange             184
Kern               160
Fresno             141
Santa Clara  

## Step 12: Check for Racial Disparities (Preliminary)

In [47]:
# Mean sentence by ethnicity
analysis_data_clean.groupby('ethnicity')['aggregate sentence in months'].agg(['mean', 'median', 'count'])

,mean,median,count
ethnicity,,,
American Indian,182.000000,164.0,42
Black,218.530806,180.0,1055
Cambodian,124.000000,124.0,1
Chinese,180.000000,180.0,4
Cuban,238.000000,238.0,2
Filipino,139.571429,90.0,28
Guatemalan,300.000000,300.0,1
Hawaiian,144.000000,144.0,1
Hispanic,180.974707,144.0,1621


In [48]:
# Mean suitability score by ethnicity
analysis_data_clean.groupby('ethnicity')['score'].agg(['mean', 'median'])

,mean,median
ethnicity,,
American Indian,0.347944,0.000000
Black,0.283570,0.000000
Cambodian,0.833333,0.833333
Chinese,0.000000,0.000000
Cuban,0.250000,0.250000
Filipino,0.314935,0.000000
Guatemalan,0.000000,0.000000
Hawaiian,0.000000,0.000000
Hispanic,0.239591,0.000000


## Step 13: Save Analysis-Ready Dataset

In [49]:
# Save to CSV
output_path = outputs_dir / "regression_analysis_data.csv"
analysis_data_clean.to_csv(output_path, index=False)

In [50]:
print(f"Saved: {output_path}")
print(f"Rows: {len(analysis_data_clean):,}")
print(f"Columns: {len(analysis_data_clean.columns)}")

Saved: ..\outputs\regression_analysis_data.csv
Rows: 4,165
Columns: 19


## Step 14: Export Column List for Reference

In [51]:
# List all columns in final dataset
print("Columns in regression_analysis_data.csv:")
for col in analysis_data_clean.columns:
    print(f"  - {col}")

Columns in regression_analysis_data.csv:
  - cdcno
  - desc_nonvio_curr
  - score
  - score_out_of
  - desc_nonvio_past
  - severity_trend
  - ethnicity
  - aggregate sentence in months
  - controlling case sentencing county
  - offense category_demo
  - time served in years
  - offense_clean
  - offense description
  - offense category_offense
  - offense_table
  - offense_type
  - high_suitability
  - violent_offense
  - suitability_category


## Summary

**Dataset Created:** `regression_analysis_data.csv`

**Key Variables:**
- **Outcome:** `aggregate sentence in months`
- **Main Predictor:** `ethnicity`
- **Controls:** `score`, `offense_table`, `controlling case sentencing county`
- **Individual Metrics:** `desc_nonvio_curr`, `desc_nonvio_past`, `severity_trend`

**Ready for:**
- Multiple Linear Regression (Model 1, 2, 3)
- Interaction analysis